In [ ]:
#build training data from the folds 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import os

'''
2. You’ll find 50 folds of the data in the `training/uploads /` directory.
1. Here, you are expected to build the data loader using `pd.concat` and looping.
'''

dir = 'data/training/uploads/'
files = os.listdir(dir)
files.sort()
print(files)

df = pd.DataFrame()
for file in files:
    df = pd.concat([df, pd.read_csv(dir+file)])
    print(df.shape)

df.head()

#save df to a csv
df.to_csv('data/df.csv', index=False)



In [ ]:
df.shape

In [ ]:
# 3. Once, you are done with this, you will be expected to build multiple models and train it using the training data generated from the previous step.

# 4. Next, you will be expected to get the predictions of the test data, in the expected format. The submission file contains:
# 1. `machine_id`: An identifier to identify the machines
# 2. `malware_status`: The output of the model(which is expected to be either 0 or 1).


In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import sys
import io

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import scikitplot as skplt
import scipy as sp


In [2]:
#load data
df = pd.read_csv('data/train.csv')
df.head()

/var/folders/kh/vjrj1xvj5jd2tsy0w3350xzw0000gn/T/ipykernel_59990/1738660457.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/train.csv')


,machine_id,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,0,win8defender,1.1.15100.1,4.18.1806.18062,1.273.493.0,0,7.0,0,NaN,53447.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,31.713231,Male
1,1,win8defender,1.1.15200.1,4.18.1807.18075,1.275.36.0,0,7.0,0,NaN,53447.0,...,NaN,0.0,0,0,0.0,1.0,11.0,0,40.099499,Female
2,2,win8defender,1.1.15200.1,4.18.1807.18075,1.275.26.0,0,7.0,0,NaN,53447.0,...,NaN,0.0,0,0,0.0,1.0,15.0,0,35.350487,Female
3,3,win8defender,1.1.15100.1,4.18.1806.18062,1.273.241.0,0,7.0,0,NaN,53447.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,42.771937,Male
4,4,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1749.0,0,7.0,0,NaN,53447.0,...,NaN,0.0,0,0,0.0,0.0,10.0,0,40.323890,Male


In [3]:
df = df.drop(['machine_id'], axis=1)
df.head()

,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,av_prod_installed,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,win8defender,1.1.15100.1,4.18.1806.18062,1.273.493.0,0,7.0,0,NaN,53447.0,1.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,31.713231,Male
1,win8defender,1.1.15200.1,4.18.1807.18075,1.275.36.0,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,1.0,11.0,0,40.099499,Female
2,win8defender,1.1.15200.1,4.18.1807.18075,1.275.26.0,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,1.0,15.0,0,35.350487,Female
3,win8defender,1.1.15100.1,4.18.1806.18062,1.273.241.0,0,7.0,0,NaN,53447.0,1.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,42.771937,Male
4,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1749.0,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,0.0,10.0,0,40.323890,Male


In [56]:
print("The dataset contains columns of the following data types : \n" +
      str(df.dtypes))
print("The dataset contains following number of records for each of the columns : \n" +
      str(df.count()))


The dataset contains columns of the following data types : 
security_product_name     object
machine_version           object
app_version               object
av_sig_version            object
is_beta                    int64
                          ...   
is_gamer                 float64
region_id                float64
malware_status             int64
age                      float64
sex                       object
Length: 84, dtype: object
The dataset contains following number of records for each of the columns : 
security_product_name    5000000
machine_version          5000000
app_version              5000000
av_sig_version           5000000
is_beta                  5000000
                          ...   
is_gamer                 4829346
region_id                4829346
malware_status           5000000
age                      5000000
sex                      5000000
Length: 84, dtype: int64


In [4]:
#label encode across the dataset to remove string/object values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])
df.head()

,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,av_prod_installed,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,4,64,51,7758,0,7.0,0,NaN,53447.0,1.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,31.713231,1
1,4,65,57,8133,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,1.0,11.0,0,40.099499,0
2,4,65,57,8105,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,1.0,15.0,0,35.350487,0
3,4,64,51,7696,0,7.0,0,NaN,53447.0,1.0,...,0.0,0.0,0,0,0.0,0.0,10.0,1,42.771937,1
4,4,64,57,7669,0,7.0,0,NaN,53447.0,1.0,...,NaN,0.0,0,0,0.0,0.0,10.0,0,40.323890,1


In [5]:
#impute missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df.head()

,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,av_prod_installed,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,4.0,64.0,51.0,7758.0,0.0,7.0,0.0,239.0,53447.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,31.713231,1.0
1,4.0,65.0,57.0,8133.0,0.0,7.0,0.0,239.0,53447.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0,40.099499,0.0
2,4.0,65.0,57.0,8105.0,0.0,7.0,0.0,239.0,53447.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,15.0,0.0,35.350487,0.0
3,4.0,64.0,51.0,7696.0,0.0,7.0,0.0,239.0,53447.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,42.771937,1.0
4,4.0,64.0,57.0,7669.0,0.0,7.0,0.0,239.0,53447.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,40.323890,1.0


In [6]:
#change all float values to int
df = df.astype(int)
df.head()

,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,av_prod_installed,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,4,64,51,7758,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,1,31,1
1,4,65,57,8133,0,7,0,239,53447,1,...,0,0,0,0,0,1,11,0,40,0
2,4,65,57,8105,0,7,0,239,53447,1,...,0,0,0,0,0,1,15,0,35,0
3,4,64,51,7696,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,1,42,1
4,4,64,57,7669,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,0,40,1


In [7]:
#defining categorical columns
categoricalColumns = ['security_product_name',
                      'platform', 'processor', 'os_ptfm_sub_release', 'os_build_lab', 'sku_edition', 'smart_screen', 'mdc2_form_factor', 'device_family',
                      'disk_type', 'chassis_type', 'power_profile', 'battery_type', 'architecture', 'branch', 'edition', 'sku', 'install_type_name', 'auto_update_opt', 'state_name', 'activation_channel', 'flight_ring', 'sex']

impute_categorical = SimpleImputer(strategy="most_frequent")
onehot_categorical = OneHotEncoder(handle_unknown='ignore')
categorical_transformer = Pipeline(
    steps=[('impute', impute_categorical), ('onehot', onehot_categorical)])


In [8]:
numericalColumns = df.select_dtypes(
    include=[np.float, np.int]).columns

print("Numerical columns : ")
print(numericalColumns)

scaler_numerical = StandardScaler()

numerical_transformer = Pipeline(steps=[('scale', scaler_numerical)])


/var/folders/kh/vjrj1xvj5jd2tsy0w3350xzw0000gn/T/ipykernel_59990/177523717.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  include=[np.float, np.int]).columns
/var/folders/kh/vjrj1xvj5jd2tsy0w3350xzw0000gn/T/ipykernel_59990/177523717.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance:

Numerical columns : 
Index(['security_product_name', 'machine_version', 'app_version',
       'av_sig_version', 'is_beta', 'rtp_state', 'is_sxs_passive',
       'default_browser', 'av_status', 'av_prod_installed', 'av_prod_enabled',
       'has_tpm', 'country_id', 'city_id', 'org_id', 'geo_id', 'local_name_id',
       'platform', 'processor', 'os_version', 'os_build', 'os_suite',
       'os_ptfm_sub_release', 'os_build_lab', 'sku_edition', 'is_protected',
       'auto_sample_opt_in', 'pua_mode', 's_mode', 'ie_ver_id', 'smart_screen',
       'firewall', 'uac_luaenable', 'mdc2_form_factor', 'device_family',
       'name_id', 'model_id', 'core_count_processer', 'manufacturer_id',
       'model_id.1', 'processor_class', 'disk_capacity', 'disk_type',
       'system_volume_capacity', 'has_optical_drive', 'ram_capacity',
       'chassis_type', 'diagonal_display_size',
       'display_resolution_horizontal', 'display_resolution_vertical',
       'power_profile', 'battery_type', 'number_of_char

In [9]:
preprocessorForCategoricalColumns = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categoricalColumns)])

preprocessorForAllColumns = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categoricalColumns),
                    ('num', numerical_transformer, numericalColumns)])

#if column is not found in the list of categorical columns, skip it



In [10]:
#ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

df_temp = preprocessorForCategoricalColumns.fit_transform(df)
print("Data after transforming :")
print(df_temp)

df_temp_2 = preprocessorForAllColumns.fit_transform(df)
print("Data after transforming :")
print(df_temp_2)


Data after transforming :
  (0, 4)	1.0
  (0, 6)	1.0
  (0, 11)	1.0
  (0, 16)	1.0
  (0, 269)	1.0
  (0, 643)	1.0
  (0, 659)	1.0
  (0, 676)	1.0
  (0, 682)	1.0
  (0, 684)	1.0
  (0, 724)	1.0
  (0, 742)	1.0
  (0, 787)	1.0
  (0, 815)	1.0
  (0, 824)	1.0
  (0, 847)	1.0
  (0, 879)	1.0
  (0, 911)	1.0
  (0, 917)	1.0
  (0, 921)	1.0
  (0, 927)	1.0
  (0, 937)	1.0
  (0, 942)	1.0
  (1, 4)	1.0
  (1, 6)	1.0
  :	:
  (4999998, 937)	1.0
  (4999998, 942)	1.0
  (4999999, 4)	1.0
  (4999999, 6)	1.0
  (4999999, 11)	1.0
  (4999999, 17)	1.0
  (4999999, 279)	1.0
  (4999999, 643)	1.0
  (4999999, 667)	1.0
  (4999999, 676)	1.0
  (4999999, 682)	1.0
  (4999999, 684)	1.0
  (4999999, 724)	1.0
  (4999999, 742)	1.0
  (4999999, 814)	1.0
  (4999999, 815)	1.0
  (4999999, 826)	1.0
  (4999999, 847)	1.0
  (4999999, 879)	1.0
  (4999999, 910)	1.0
  (4999999, 916)	1.0
  (4999999, 921)	1.0
  (4999999, 927)	1.0
  (4999999, 937)	1.0
  (4999999, 941)	1.0
Data after transforming :
  (0, 4)	1.0
  (0, 6)	1.0
  (0, 11)	1.0
  (0, 16)	1.0
  (0

In [11]:
df.head()

,security_product_name,machine_version,app_version,av_sig_version,is_beta,rtp_state,is_sxs_passive,default_browser,av_status,av_prod_installed,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,4,64,51,7758,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,1,31,1
1,4,65,57,8133,0,7,0,239,53447,1,...,0,0,0,0,0,1,11,0,40,0
2,4,65,57,8105,0,7,0,239,53447,1,...,0,0,0,0,0,1,15,0,35,0
3,4,64,51,7696,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,1,42,1
4,4,64,57,7669,0,7,0,239,53447,1,...,0,0,0,0,0,0,10,0,40,1


In [12]:
df.shape

(5000000, 84)

In [15]:
#drop the columns with more than 60% 0 values
threshold = 0.6
df = df.loc[:, df.apply(lambda x: (x == 0).mean()) < threshold]
df.head()

#split the data into train and test

,security_product_name,machine_version,app_version,av_sig_version,rtp_state,default_browser,av_status,av_prod_installed,av_prod_enabled,has_tpm,...,state_name,activation_channel,flight_ring,firmware_manufacturer_id,firmware_version_id,secure_boot_status,region_id,malware_status,age,sex
0,4,64,51,7758,7,1658,53447,1,1,1,...,1,2,6,897,7221,1,10,1,31,1
1,4,65,57,8133,7,1658,53447,1,1,1,...,1,2,6,355,19951,0,11,0,40,0
2,4,65,57,8105,7,1658,53447,1,1,1,...,1,2,6,142,8036,0,15,0,35,0
3,4,64,51,7696,7,1658,53447,1,1,1,...,1,0,6,554,33097,1,10,1,42,1
4,4,64,57,7669,7,1658,53447,1,1,1,...,1,0,6,554,33084,1,10,0,40,1


In [15]:
df.shape

(5000000, 70)

In [13]:
#drop columns with high correlation
corr = df.corr()
corr_list = []
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.8:
            colname = corr.columns[i]
            corr_list.append(colname)
print(corr_list)

df = df.drop(corr_list, axis=1)
df.head()

['av_sig_version', 'is_sxs_passive', 'has_tpm', 'os_version', 'sku_edition', 'ie_ver_id', 'ie_ver_id', 'ie_ver_id', 'display_resolution_vertical', 'version', 'architecture', 'branch', 'build_number', 'build_number', 'edition', 'edition', 'sku', 'sku', 'sku', 'ui_locale_identifier']


,security_product_name,machine_version,app_version,is_beta,rtp_state,default_browser,av_status,av_prod_installed,av_prod_enabled,country_id,...,wim_boot_status,virtual_dev_status,touch_support,pen_support,aoac_support,is_gamer,region_id,malware_status,age,sex
0,4,64,51,0,7,239,53447,1,1,141,...,0,0,0,0,0,0,10,1,31,1
1,4,65,57,0,7,239,53447,1,1,100,...,0,0,0,0,0,1,11,0,40,0
2,4,65,57,0,7,239,53447,1,1,60,...,0,0,0,0,0,1,15,0,35,0
3,4,64,51,0,7,239,53447,1,1,141,...,0,0,0,0,0,0,10,1,42,1
4,4,64,57,0,7,239,53447,1,1,44,...,0,0,0,0,0,0,10,0,40,1


In [14]:
#check for null values
df.isnull().sum()


security_product_name    0
machine_version          0
app_version              0
is_beta                  0
rtp_state                0
                        ..
is_gamer                 0
region_id                0
malware_status           0
age                      0
sex                      0
Length: 70, dtype: int64

In [15]:
# remove nan values from df_temp_2
df_temp_2 = pd.DataFrame(df_temp_2)
df_temp_2 = df_temp_2.dropna()
df_temp_2.head()

,0
0,"(0, 4)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
1,"(0, 4)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
2,"(0, 4)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
3,"(0, 4)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
4,"(0, 4)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."


In [16]:
features = []
features = df.drop(['malware_status'], axis=1)
label = df['malware_status']

X_train, X_test, y_train, y_test = train_test_split(
    features, label, random_state=0)
print("Dimensions of datasets that will be used for training : Input features"+str(X_train.shape) +
      " Output label" + str(y_train.shape))
print("Dimensions of datasets that will be used for testing : Input features"+str(X_test.shape) +
      " Output label" + str(y_test.shape))

Dimensions of datasets that will be used for training : Input features(3750000, 69) Output label(3750000,)
Dimensions of datasets that will be used for testing : Input features(1250000, 69) Output label(1250000,)


In [17]:
test = pd.read_csv('data/test.csv')

/var/folders/kh/vjrj1xvj5jd2tsy0w3350xzw0000gn/T/ipykernel_59990/4042438930.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('data/test.csv')


In [20]:
from sklearn.linear_model import LogisticRegression

model_name = "Logistic Regression Classifier"

logisticRegressionClassifier = LogisticRegression(
    random_state=0, multi_class='auto', solver='lbfgs', max_iter=1000)

logisticRegressionClassifier.fit(X_train, y_train)

y_pred_lrc = logisticRegressionClassifier.predict(X_test)


print("Accuracy of " + model_name + " is : " +
        str(accuracy_score(y_test, y_pred_lrc)))


Accuracy of Logistic Regression Classifier is : 0.5049968


: 

In [14]:
y_pred_lrc_test = lrc_model.predict(test)


In [19]:
type(y_pred_lrc_test)

numpy.ndarray

In [15]:
test['malware_status'] = y_pred_lrc_test
test = test[['machine_id', 'malware_status']]
test.to_csv('data/lrcn.csv', index=False)
test.head()

,machine_id,malware_status
0,5000000,0.0
1,5000001,0.0
2,5000002,0.0
3,5000003,0.0
4,5000004,0.0


In [18]:
# convert to csv with just machine_id and malware_status
test['malware_status'] = y_pred_lrc_test
test = test[['machine_id', 'malware_status']]
test.to_csv('data/lrc.csv', index=False)
test.head()

In [ ]:
#converting 
def transform(self, X):
    import scipy.sparse

    is_sparse = scipy.sparse.issparse(X)
    if self.preprocessor is None:
        raise NotImplementedError()
    X = self.preprocessor.transform(X)
    if is_sparse:
        return X
    elif isinstance(X, np.ndarray):
        return X
    else:
        return X.toarray()



In [40]:
from sklearn.naive_bayes import GaussianNB

model_name = 'Naive Bayes Classifier'

nbClassifier = GaussianNB()

nb_model = Pipeline(steps=[('preprocessor', preprocessorForCategoricalColumns),
                            ('classifier', nbClassifier)])
                            

nb_model.fit(X_train.values.scipy.toarray(), y_train)

y_pred_nb = nb_model.predict(X_test.values.toarray())

print("Accuracy of " + model_name + " is : " + str(accuracy_score(y_test, y_pred_nb)))


AttributeError: 'numpy.ndarray' object has no attribute 'scipy'

In [ ]:
from sklearn.

In [23]:
#cart model
from sklearn.tree import DecisionTreeClassifier

model_name = 'Decision Tree Classifier'

cartClassifier = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None
)

cart_model = Pipeline(steps=[('preprocessor', preprocessorForCategoricalColumns),('classifier', cartClassifier)])

cart_model.fit(X_train,y_train)

y_pred_cart= cart_model.predict(X_test)

print("Accuracy of " + model_name + " is : " + str(accuracy_score(y_test, y_pred_cart)))

Accuracy of Decision Tree Classifier is : 0.5836992


In [21]:
y_pred_cart_test = cart_model.predict(test)

NameError: name 'cart_model' is not defined

In [28]:
y_pred_cart_test.shape

(3921483,)

In [33]:
#convert to csv with just machine_id and malware_status
test['malware_status'] = y_pred_cart_test
test = test[['machine_id', 'malware_status']]
test.to_csv('data/dt.csv', index=False)

In [ ]:
#remove columns with all null values
#df = df.dropna(axis=1, how='any')
df.shape

In [43]:
df.columns

Index(['security_product_name', 'machine_version', 'app_version',
       'av_sig_version', 'is_beta', 'rtp_state', 'is_sxs_passive',
       'default_browser', 'av_status', 'av_prod_installed', 'av_prod_enabled',
       'has_tpm', 'country_id', 'city_id', 'org_id', 'geo_id', 'local_name_id',
       'platform', 'processor', 'os_version', 'os_build', 'os_suite',
       'os_ptfm_sub_release', 'os_build_lab', 'sku_edition', 'is_protected',
       'auto_sample_opt_in', 'pua_mode', 's_mode', 'ie_ver_id', 'smart_screen',
       'firewall', 'uac_luaenable', 'mdc2_form_factor', 'device_family',
       'name_id', 'model_id', 'core_count_processer', 'manufacturer_id',
       'model_id.1', 'processor_class', 'disk_capacity', 'disk_type',
       'system_volume_capacity', 'has_optical_drive', 'ram_capacity',
       'chassis_type', 'diagonal_display_size',
       'display_resolution_horizontal', 'display_resolution_vertical',
       'power_profile', 'battery_type', 'number_of_charges_for_battery',
   

In [44]:
#check for null values
df.isnull().sum()

security_product_name    0
machine_version          0
app_version              0
av_sig_version           0
is_beta                  0
                        ..
is_gamer                 0
region_id                0
malware_status           0
age                      0
sex                      0
Length: 84, dtype: int64

In [42]:
#df = df.dropna()
df.shape

(5000000, 84)

In [ ]:
#for all malware_status = 1, find the common attributes
print(df[df['malware_status']==1].describe())

#check for class imbalance
print(df['malware_status'].value_counts())

In [ ]:
#find correlation between features
corr = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, annot=True, cmap='coolwarm')
#plt.show()

In [ ]:
#note all the features with correlation greeater than 0.1
corr[corr['malware_status']>0.1]['malware_status']


In [ ]:
type(corr)

In [ ]:
#from corr dataframe, extract the feature pairs with correlation greater than 0.1
corr_features = []
for i in corr.index:
    for j in corr.columns:
        if corr[i][j] > 0.35 and corr[i][j] < 1:
            corr_features.append((i,j))
corr_features


In [45]:
#malware_status is highly correlated with the following features
#1. 'av_prod_installed
#2. 'av_status' 

#drop the features with high correlation to avoid multicollinearity for 'malware_status'
df = df.drop(['av_prod_installed', 'av_status'], axis=1)
df.shape

(5000000, 82)

In [ ]:
#drop columns having more than 50% null values
df = df.dropna(axis=1, thresh=0.5*df.shape[0])
df.shape


In [ ]:
#print column with null values present
df.columns[df.isnull().any()]


In [46]:
#drop columns with null values minus av_status

df = df.drop(['rtp_state',
              'city_id', 'org_id', 'geo_id', 'os_build_lab', 's_mode',
              'ie_ver_id', 'smart_screen', 'uac_luaenable', 'name_id',
              'model_id', 'core_count_processer', 'manufacturer_id', 'model_id.1',
              'disk_capacity', 'disk_type', 'system_volume_capacity', 'ram_capacity',
              'chassis_type', 'diagonal_display_size',
              'display_resolution_horizontal', 'display_resolution_vertical',
              'power_profile', 'number_of_charges_for_battery',
              'install_language_identifier', 'flights_activity',
              'firmware_manufacturer_id', 'firmware_version_id', 'virtual_dev_status',
              'aoac_support', 'is_gamer', 'region_id'], axis=1)
df.shape



(5000000, 50)

In [ ]:
#impute null values with mean, median or mode
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna(df[col].mean())
df.isnull().sum()

In [ ]:
# final data clean up
df = df.drop(['machine_id'], axis=1)
df.shape

In [ ]:
#encoding categorical features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])
df.head()

In [ ]:
df.shape

In [ ]:
#load test data
test = pd.read_csv('data/test.csv')
test.head()

In [ ]:
#use stratified sampling to split the data into train and test
X_train = df.drop(['malware_status'], axis=1)
y_train = df[['malware_status']]
X_train.shape, y_train.shape, test.shape



model = RandomForestClassifier(n_estimators=1000, random_state=42, min_samples_split=5,
                               min_samples_leaf=2, max_features='sqrt', max_depth=10, bootstrap=True)

model.fit(X_train, y_train)

y_pred = model.predict(test)


#check accuracy

print(accuracy_score(y_train, model.predict(X_train)))

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
gg = pd.read_csv('cpp_tonight__.csv')
gg["malware_status"] = y_pred
gg.head()

In [ ]:
gg["malware_status"] = y_pred